In [12]:

import csv
import json
import pandas as pd
import os
from datetime import datetime
from pytz import timezone
import matplotlib.pyplot as plt
import datetime
import urllib
import numpy as np
import plotly.graph_objects as go



In [13]:
# load the two csv files
raw_df = pd.read_csv("raw_data/raw_politcal_posts_three.csv")
print(raw_df.shape)


raw_df2 = pd.read_csv("raw_data/raw_politcal_posts_two.csv")
print(raw_df2.shape)

#combine and remove duplicates
all_df = pd.concat([raw_df, raw_df2])
all_df = all_df.drop_duplicates(subset=['id'])
print(all_df.shape)


(2434, 9)
(2413, 9)
(2423, 9)


In [14]:
# columns subreddit
subreddits = all_df['subreddit'].unique()
print(subreddits)
print(len(subreddits))

['Libertarian' 'Anarchism' 'socialism' 'progressive' 'Conservative'
 'americanpirateparty' 'democrats' 'Liberal' 'Republican'
 'LibertarianLeft' 'Liberty' 'Anarcho_Capitalism' 'alltheleft' 'neoprogs'
 'labor' 'blackflag' 'GreenParty' 'democracy' 'IWW' 'Marxism'
 'Objectivism' 'LibertarianSocialism' 'Capitalism' 'feminisms'
 'republicans' 'Egalitarianism' 'anarchafeminism' 'SocialDemocracy'
 'Postleftanarchism' 'AnarchoPacifism' 'conservatives' 'voluntarism'
 'PirateParty' 'Anarchist' 'Communist']
35


In [17]:
# Republican, republicans 
all_df.loc[all_df['subreddit'] == 'republicans', 'subreddit'] = 'Republican'

# Anarchism, Anarchist
all_df.loc[all_df['subreddit'] == 'Anarchism', 'subreddit'] = 'Anarchist'

# americanpirateparty, PirateParty
all_df.loc[all_df['subreddit'] == 'americanpirateparty', 'subreddit'] = 'PirateParty'

subred_rows = all_df['subreddit'].unique()
print(subred_rows)
print(len(subred_rows))

['Libertarian' 'Anarchist' 'socialism' 'progressive' 'Conservative'
 'PirateParty' 'democrats' 'Liberal' 'Republican' 'LibertarianLeft'
 'Liberty' 'Anarcho_Capitalism' 'alltheleft' 'neoprogs' 'labor'
 'blackflag' 'GreenParty' 'democracy' 'IWW' 'Marxism' 'Objectivism'
 'LibertarianSocialism' 'Capitalism' 'feminisms' 'Egalitarianism'
 'anarchafeminism' 'SocialDemocracy' 'Postleftanarchism' 'AnarchoPacifism'
 'conservatives' 'voluntarism' 'Communist']
32


In [19]:
# generate how many post exist for each subreddit
subred_count = all_df['subreddit'].value_counts()
print(subred_count)


Conservative            220
Libertarian             206
Anarcho_Capitalism      204
Anarchist               203
socialism               202
Republican              173
conservatives           127
democrats               122
progressive             115
Liberal                 112
GreenParty              112
alltheleft              111
SocialDemocracy          98
Capitalism               87
Objectivism              60
blackflag                47
Marxism                  39
feminisms                35
labor                    32
democracy                27
LibertarianSocialism     21
IWW                      18
LibertarianLeft          14
Egalitarianism           10
Postleftanarchism         6
PirateParty               6
neoprogs                  5
AnarchoPacifism           4
Liberty                   3
anarchafeminism           2
Communist                 1
voluntarism               1
Name: subreddit, dtype: int64


In [21]:
# filter out subreddits with less than 50 posts from the main dataframe
all_df = all_df[all_df['subreddit'].isin(subred_count.index[subred_count > 50])]
subred_count = subred_count[subred_count > 50]
print(subred_count)
print(all_df.shape)

Conservative          220
Libertarian           206
Anarcho_Capitalism    204
Anarchist             203
socialism             202
Republican            173
conservatives         127
democrats             122
progressive           115
Liberal               112
GreenParty            112
alltheleft            111
SocialDemocracy        98
Capitalism             87
Objectivism            60
Name: subreddit, dtype: int64
(2152, 9)


Creating the future df to add things to 

In [22]:
#make a df that has a claim column and if text or title
df_one = pd.DataFrame(columns=['claim', 'type', 'subreddit'])


In [23]:
# This contains purely text that is good to analyze

#drop NaN from selftext
text_df = all_df.dropna(subset=['selftext'])

#make it into a list and split it by space
text_df['selftext'] = text_df['selftext'].apply(lambda x: x.split())


print(text_df['selftext'].head())
print(text_df.shape)


0     [It’s, coming, so, we, got, to, be, prepared, ...
4     [I, consider, myself, a, believer, in, what, i...
6     [Just, like, the, title, says,, I, wonder, why...
7     [Do, you, believe, humanity, taking, collectiv...
10    [I, hear, in, the, media, on, how, the, climat...
Name: selftext, dtype: object
(657, 9)


/Users/liviagimenes/cs1951a_venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [24]:
#insert title_list into claim and set all of the type to title
for index, row in text_df.iterrows():
    df_one = df_one.append({'claim': row['selftext'], 'type': 'text', 'subreddit': row['subreddit']}, ignore_index=True)
#drop null rows
df_one = df_one.dropna(subset=['claim'])

print(df_one.head())
print(df_one.shape)

                                               claim  type    subreddit
0  [It’s, coming, so, we, got, to, be, prepared, ...  text  Libertarian
1  [I, consider, myself, a, believer, in, what, i...  text  Libertarian
2  [Just, like, the, title, says,, I, wonder, why...  text  Libertarian
3  [Do, you, believe, humanity, taking, collectiv...  text  Libertarian
4  [I, hear, in, the, media, on, how, the, climat...  text  Libertarian
(657, 3)


In [25]:
# this is trying to get titles that might be good to analyze
title_df = all_df.dropna(subset=['title'])


#drop duplicates
title_df = title_df.drop_duplicates(subset=['title'])
print(title_df.head())
print(title_df.shape)

#get all of the titles into a list 
title_df['title'] = title_df['title'].apply(lambda x: x.split())
print(title_df['title'])

#filter the list to not include list that have less than 15 words
# create boolean mask based on title length
mask = title_df['title'].apply(lambda x: len(' '.join(x)) > 15)

# filter dataframe based on boolean mask
df = title_df[mask]



print(title_df['title'])




        id    subreddit                                              title  \
0  10w1dj4  Libertarian  What is the Libertarian solution to climate ch...   
1    aocqp  Libertarian  Prof Anastasios Tsonis: University of Wisconsi...   
2   v65fv1  Libertarian  Three Supreme Court cases to watch beyond abor...   
3   voask2  Libertarian  Supreme Court limits EPA’s power to combat cli...   
4   uqcz0p  Libertarian  A green libertarian perspective: is bad to hav...   

              author                                                url  \
0  ErgonomicSikorsky  https://www.reddit.com/r/Libertarian/comments/...   
1          DougDante  http://www.dailymail.co.uk/sciencetech/article...   
2           Elranzer  https://www.brennancenter.org/our-work/researc...   
3   Careless_Bat2543  https://www.washingtonpost.com/politics/2022/0...   
4           cr7fan89  https://www.reddit.com/r/Libertarian/comments/...   

                                            selftext  score  num_comments  \
0  

In [26]:

df_two = pd.DataFrame(columns=['claim','type',"subreddit"])
for index, row in text_df.iterrows():
    df_two = df_two.append({'claim': row['selftext'], 'type': 'title', 'subreddit': row['subreddit']}, ignore_index=True)
df_two = df_two.dropna(subset=['claim'])

print(df_two.head())
print(df_two.shape)

#combine the two df
df = pd.concat([df_one, df_two])
print(df.head())
print(df.shape)

                                               claim   type    subreddit
0  [It’s, coming, so, we, got, to, be, prepared, ...  title  Libertarian
1  [I, consider, myself, a, believer, in, what, i...  title  Libertarian
2  [Just, like, the, title, says,, I, wonder, why...  title  Libertarian
3  [Do, you, believe, humanity, taking, collectiv...  title  Libertarian
4  [I, hear, in, the, media, on, how, the, climat...  title  Libertarian
(657, 3)
                                               claim  type    subreddit
0  [It’s, coming, so, we, got, to, be, prepared, ...  text  Libertarian
1  [I, consider, myself, a, believer, in, what, i...  text  Libertarian
2  [Just, like, the, title, says,, I, wonder, why...  text  Libertarian
3  [Do, you, believe, humanity, taking, collectiv...  text  Libertarian
4  [I, hear, in, the, media, on, how, the, climat...  text  Libertarian
(1314, 3)


In [27]:
#save the data
df.to_csv("clean/political_posts.csv", index=False)